<a href="https://colab.research.google.com/github/mitran27/GenerativeNetworks/blob/main/Dreambooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Oct  3 07:12:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install bitsandbytes
!pip install accelerate
!pip install diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.8 MB/s eta 0:00:00


In [3]:

from PIL import Image

def image_grid(imgs, rows, cols, resize=256):
    assert len(imgs) == rows * cols

    if resize is not None:
        imgs = [img.resize((resize, resize)) for img in imgs]

    w, h = imgs[0].size
    grid_w, grid_h = cols * w, rows * h
    grid = Image.new("RGB", size=(grid_w, grid_h))

    for i, img in enumerate(imgs):
        x = i % cols * w
        y = i // cols * h
        grid.paste(img, box=(x, y))

    return grid

In [ ]:
# prompt: clone diffusers repo

!git clone https://github.com/huggingface/diffusers.git


Cloning into 'diffusers'...
remote: Enumerating objects: 70436, done.
remote: Counting objects: 100% (9752/9752), done.
remote: Compressing objects: 100% (1294/1294), done.
remote: Total 70436 (delta 9163), reused 8682 (delta 8363), pack-reused 60684 (from 1)
Receiving objects: 100% (70436/70436), 49.16 MiB | 9.79 MiB/s, done.
Resolving deltas: 100% (52193/52193), done.


In [8]:
!unzip /content/dreambooth_images.zip

Archive:  /content/dreambooth_images.zip
   creating: prior/
  inflating: prior/generated_0.png   
  inflating: prior/generated_1.png   
  inflating: prior/generated_10.png  
  inflating: prior/generated_101.png  
  inflating: prior/generated_104.png  
  inflating: prior/generated_105.png  
  inflating: prior/generated_106.png  
  inflating: prior/generated_107.png  
  inflating: prior/generated_108.png  
  inflating: prior/generated_11.png  
  inflating: prior/generated_110.png  
  inflating: prior/generated_111.png  
  inflating: prior/generated_112.png  
  inflating: prior/generated_115.png  
  inflating: prior/generated_12.png  
  inflating: prior/generated_129.png  
  inflating: prior/generated_13.png  
  inflating: prior/generated_130.png  
  inflating: prior/generated_131.png  
  inflating: prior/generated_133.png  
  inflating: prior/generated_134.png  
  inflating: prior/generated_135.png  
  inflating: prior/generated_136.png  
  inflating: prior/generated_138.png  
  inflati

In [1]:
import os
from PIL import Image
from tqdm import tqdm

In [2]:
MODEL_NAME = "stabilityai/stable-diffusion-2"
PRECISION = "fp16"
instance_prompt = 'A photo of a qswuejriy'
class_prompt = 'A photo of a a woman'

In [3]:
from transformers import AutoTokenizer, PretrainedConfig
from transformers import CLIPTextModel
from torch.utils.data import Dataset
from torchvision import transforms
from diffusers.optimization import get_scheduler
import bitsandbytes as bnb
from accelerate.utils import ProjectConfiguration, set_seed
from accelerate import Accelerator
import torch
import torch.nn.functional as F
import diffusers
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    DiffusionPipeline,
    StableDiffusionPipeline,
    UNet2DConditionModel,
)


In [4]:
class DreamBoothDataset(Dataset):
  def __init__(self,
               Instance_dir,
               instance_prompt,
               tokenizer,
               class_dir,
               class_prompt,
               img_size=512,
               tokenizer_max_length=None) -> None:

          self.tokenizer = tokenizer
          self.instance_dir = Instance_dir
          self.instance_prompt = instance_prompt
          self.class_dir = class_dir
          self.class_prompt = class_prompt
          self.img_size = img_size
          self.tokenizer_max_length = tokenizer_max_length

          self.instance_images = os.listdir(self.instance_dir)
          self.class_images = os.listdir(self.class_dir)

          self._prepare_transforms();

          self._prepare_instance_tokens()
          self._prepare_class_tokens()

  def __len__(self):
        return len(self.class_images)

  def _prepare_transforms(self):
        self._image_transforms = transforms.Compose(
            [
                transforms.Resize(self.img_size),
                transforms.CenterCrop(self.img_size),
                transforms.ToTensor(),
                transforms.Normalize([0.5], [0.5]),
            ]
        )
  def _prepare_instance_tokens(self):
      max_length = self.tokenizer.model_max_length if self.tokenizer_max_length is None else self.tokenizer_max_length
      instance_tokens = self.tokenizer(
          self.instance_prompt,
          truncation=True,
          padding="max_length",
          max_length=max_length,
          return_tensors="pt",
      )

      self.instance_data = {
          "instance_prompt_ids": instance_tokens["input_ids"],
          "instance_attention_mask": instance_tokens["attention_mask"],
      }

  def _prepare_class_tokens(self):
        max_length = self.tokenizer.model_max_length if self.tokenizer_max_length is None else self.tokenizer_max_length
        class_tokens = self.tokenizer(
          self.class_prompt,
          truncation=True,
          padding="max_length",
          max_length=max_length,
          return_tensors="pt",
      )

        self.class_data = {
          "class_prompt_ids": class_tokens["input_ids"],
          "aclss_ttention_mask": class_tokens["attention_mask"],
      }

  def pre_compute_embedding(self, text_encoder):


    with torch.no_grad():

      self.instance_data["instance_embedding"] = text_encoder(
          input_ids=self.instance_data["instance_prompt_ids"].to(text_encoder.device),
          attention_mask=self.instance_data["instance_attention_mask"].to(text_encoder.device),
          return_dict = False
      )[0].detach()

      self.class_data["class_embedding"] = text_encoder(
          input_ids=self.class_data["class_prompt_ids"].to(text_encoder.device),
          attention_mask=self.class_data["aclss_ttention_mask"].to(text_encoder.device),
          return_dict = False
      )[0]



  def __getitem__(self, index) :
        instance_image = Image.open(os.path.join(self.instance_dir, self.instance_images[index % len(self.instance_images)]))
        class_image = Image.open(os.path.join(self.class_dir, self.class_images[index]))

        assert class_image.mode == "RGB" and instance_image.mode == "RGB"

        data = {}

        data["instance_image"] = self._image_transforms(instance_image)
        data["class_image"] = self._image_transforms(class_image)

        data.update(self.instance_data)
        data.update(self.class_data)

        return data


In [5]:
def merge_class_instance(batch):
  merged_batch = {}

  images = [data["instance_image"] for data in batch]
  latents = [data["instance_embedding"] for data in batch]

  images += [data["class_image"] for data in batch]
  latents += [data["class_embedding"] for data in batch]
  merged_batch["images"] = torch.stack(images).to(memory_format=torch.contiguous_format).float()
  merged_batch["latents"] = torch.cat(latents,dim=0)

  return merged_batch






In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if PRECISION=="fp16" else torch.float32
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, subfolder="tokenizer",variant=PRECISION)
text_encoder = CLIPTextModel.from_pretrained(MODEL_NAME, subfolder="text_encoder", variant=PRECISION, torch_dtype=dtype).to(device)
text_encoder.requires_grad_(False)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 1024)
      (position_embedding): Embedding(77, 1024)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-22): 23 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=True)
            (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (layer_norm2): LayerNorm((1

In [7]:
train_dataset = DreamBoothDataset("/content/instance",instance_prompt,tokenizer,"/content/prior",class_prompt)
train_dataset.pre_compute_embedding(text_encoder)

In [8]:
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=True,
        collate_fn=lambda batch: merge_class_instance(batch),
    )
for ste,batch in enumerate(train_dataloader):
  print(batch.keys(),batch['images'].shape,batch['latents'].shape)

dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents']) torch.Size([4, 3, 512, 512]) torch.Size([4, 77, 1024])
dict_keys(['images', 'latents'])

In [9]:
from abc import ABC, abstractmethod

class DLPipeline(ABC):
  def __init__(self):
    pass

  @abstractmethod
  def train(self):
    pass

  @abstractmethod
  def generate(self):
    pass

  @abstractmethod
  def save(self):
    pass

  @abstractmethod
  def load(self):
    pass

**Required Models for training Stable diffusion**

1) Tokenizer : to Convert text to input embedding

2) Text Encoder : prcess the text to latents representing the information of the context of the text

3) Scheduler : Noise and denoise the images/latents for input to the diffusion model
* Noise the input for training to predict the noise

* Denoise during inference to remove the predicted noise from input noise to process with next step

4)  VAE :encode the image to latent space

5) Unet : take the noised latent and try to predict the noise



**Training the Stable diffusion Model**

1) use tokenizer to create propt tokens and pass it to CLIP encoder to create the context embedding

2)



In [10]:
def rescale_noise_cfg(noise_cfg, noise_pred_text, guidance_rescale=0.0):
    """
    Rescale `noise_cfg` according to `guidance_rescale`. Based on findings of [Common Diffusion Noise Schedules and
    Sample Steps are Flawed](https://arxiv.org/pdf/2305.08891.pdf). See Section 3.4
    """
    std_text = noise_pred_text.std(dim=list(range(1, noise_pred_text.ndim)), keepdim=True)
    std_cfg = noise_cfg.std(dim=list(range(1, noise_cfg.ndim)), keepdim=True)
    # rescale the results from guidance (fixes overexposure)
    noise_pred_rescaled = noise_cfg * (std_text / std_cfg)
    # mix with the original results from guidance by factor guidance_rescale to avoid "plain looking" images
    noise_cfg = guidance_rescale * noise_pred_rescaled + (1 - guidance_rescale) * noise_cfg
    return noise_cfg

In [11]:
from os.path import exists
import itertools,math,random
from diffusers.image_processor import VaeImageProcessor
class DreamBoothPipeline(DLPipeline):
  def __init__(self,training_configs,
               train_clip = False,
               generate_class_images=False,
               n_class=0,
               class_images_dir=None,
               class_prompt=None,
               accelerate=False):

    print("*** Downloading and loading models ***")

    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.dtype = torch.float16 if PRECISION=="fp16" else torch.float32

    #self.tokenizer =AutoTokenizer.from_pretrained(MODEL_NAME, subfolder="tokenizer",variant=PRECISION)
    self.text_encoder = CLIPTextModel.from_pretrained(MODEL_NAME, subfolder="text_encoder", variant=PRECISION) if train_clip else None
    self.unet = UNet2DConditionModel.from_pretrained(MODEL_NAME, subfolder="unet", variant=PRECISION)
    self.vae = AutoencoderKL.from_pretrained(MODEL_NAME, subfolder="vae", variant=PRECISION)
    self.scheduler = DDPMScheduler.from_pretrained(MODEL_NAME, subfolder="scheduler")

    self.text_encoder.requires_grad_(False) if train_clip else None
    self.vae.requires_grad_(False)

    if(training_configs["gradient_checkpointing"]):
      self.unet.enable_gradient_checkpointing()
      if train_clip:
        self.text_encoder.enable_gradient_checkpointing()


    self.train_clip = train_clip
    if train_clip:
      params = itertools.chain(self.unet.parameters(), self.text_encoder.parameters())
    else:
      params = self.unet.parameters()
    self.optimizer = bnb.optim.Adam8bit(params,
                                          lr = training_configs["lr"],
                                          betas=training_configs["betas"],
                                          eps=training_configs["adam_epsilon"],
                                          weight_decay=training_configs["adam_weight_decay"])


    accelerator_project_config = ProjectConfiguration(project_dir=os.path.join(training_configs["dir"],"project"), logging_dir=os.path.join(training_configs["dir"],"log"))

    self.accelerator = Accelerator(
        gradient_accumulation_steps=training_configs["gradient_accumulation_steps"],
        mixed_precision=training_configs["mixed_precision"],
        log_with=training_configs["report_to"],
        project_config=accelerator_project_config
    )

    self.lr_scheduler = get_scheduler(
        training_configs["lr_scheduler"],
        optimizer=self.optimizer,
        num_warmup_steps=training_configs["lr_warmup_steps"] * self.accelerator.num_processes,
        num_training_steps=training_configs["max_train_steps"] * self.accelerator.num_processes,
        num_cycles=training_configs["lr_num_cycles"],
        power=training_configs["lr_power"]
    )

    if generate_class_images:
      self.generate_class_images(class_images_dir, n_class, class_prompt)

    self.accelerate = accelerate

    if(self.accelerate):
      self.unet, self.optimizer, self.lr_scheduler = self.accelerator.prepare(
            self.unet, self.optimizer, self.lr_scheduler
        )
    self.vae = self.vae.to(self.device)
    self.text_encoder = self.text_encoder.to(self.device) if train_clip else None

    self.max_train_steps = training_configs["max_train_steps"] # maximum iteration to be done

    self.vae_image_processor = VaeImageProcessor(vae_scale_factor=self.vae.config.scaling_factor)
    self.guidance_scale = 3.0


  def unwrap_model(self,model):
        model = self.accelerator.unwrap_model(model)
        return model

  def train(self, train_dataloader):
    dataloader = self.accelerator.prepare(train_dataloader)

    if(self.accelerator.mixed_precision == "fp16"):
      weight_dtype = torch.float16
    else:
      weight_dtype = torch.float32

    self.vae.to(dtype = weight_dtype)
    if self.train_clip:
      self.text_encoder.to(dtype = weight_dtype)

    epochs =  math.ceil(self.max_train_steps / len(dataloader))

    for epoch in range(epochs):
      self.unet.train()
      epoch_loss = 0
      with tqdm(total=len(dataloader), desc=f"Epoch {epoch + 1}/{epochs}", leave=False) as pbar:
        for step, batch in enumerate(dataloader):


          # Encode th image to latent vector using VAE
          #torch.Size([4, 3, 512, 512])
          image_latent = self.vae.encode(batch["images"].to(dtype = weight_dtype)).latent_dist.sample()



          image_latent = image_latent * self.vae.config.scaling_factor
          #torch.Size([4, 4, 64, 64])

          # create Noise in image
          noise = torch.randn_like(image_latent)
          #torch.Size([4, 4, 64, 64])


          # create random time steps
          timesteps = torch.randint(0, self.scheduler.config.num_train_timesteps, (image_latent.shape[0],), device=self.device, dtype=torch.long)
          #torch.Size([4])

          # Add the noise to the image latent based on the time step
          noisy_latent = self.scheduler.add_noise(image_latent, noise, timesteps)




          encoder_hidden_states = batch["latents"]
          #torch.Size([4, 77, 1024])

          pred_noise =self.unet(noisy_latent, timesteps, encoder_hidden_states)[0]  # predicts how much noise has been added from prev time step
          # torch.Size([4, 4, 64, 64])

          loss = F.mse_loss(pred_noise.float(),noise.float(),reduction="mean")  # assuming class loss and instance loss have same weightage

          epoch_loss += loss.item()

          self.accelerator.backward(loss)

          self.accelerator.clip_grad_norm_(self.unet.parameters(),1.0)

          self.optimizer.step()

          self.lr_scheduler.step()
          self.optimizer.zero_grad(set_to_none=True)

          pbar.set_postfix({"loss": loss.item(), "avg_loss": epoch_loss / (step + 1)})
          pbar.update(1)
        print(f"Epoch {epoch + 1} loss: {epoch_loss / len(dataloader)}")







  def generate(self,prompt,images_per_prompt,noise=None,diffusion_steps=50,tokenizer=None,text_encoder=None):


    unet=self.unwrap_model(self.unet)
    generator = torch.cuda.manual_seed(random.randrange(2**32 - 1))

    init_latent = torch.zeros((images_per_prompt, unet.in_channels, 512 // 8, 512 // 8), device=self.device)
    t_start = torch.tensor(0)

    self.scheduler.set_timesteps(diffusion_steps)
    noise = torch.randn(init_latent.shape, generator=generator, device=self.device)
    latents = noise * self.scheduler.init_noise_sigma

    with torch.no_grad():

      if tokenizer:
        self.tokenizer = tokenizer
      if text_encoder:
        self.text_encoder = text_encoder

      inputs = self.tokenizer([prompt]*images_per_prompt, return_tensors="pt",padding="max_length", max_length=self.tokenizer.model_max_length, truncation=True,).input_ids.to(self.device)
      text_embeddings = self.text_encoder(inputs)[0]

      uncond_inputs = self.tokenizer(
          [""] * images_per_prompt, return_tensors="pt",padding="max_length", max_length=self.tokenizer.model_max_length, truncation=True,
      ).input_ids.to(self.device)
      uncond_embeddings = self.text_encoder(uncond_inputs)[0]

      prompt_embeds = torch.cat([uncond_embeddings, text_embeddings])

      timesteps = self.scheduler.timesteps[t_start:]
      for t in self.scheduler.timesteps:
        print(t)

        latent_model_input = torch.cat([latents] * 2)
        latent_model_input = self.scheduler.scale_model_input(latent_model_input, t)

        noise_pred = unet(latent_model_input, t,prompt_embeds).sample

        noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + self.guidance_scale * (noise_pred_text - noise_pred_uncond)

        # guidance scale



        latents = self.scheduler.step(noise_pred, t, latents).prev_sample



    image = self.vae.decode(latents / self.vae.config.scaling_factor, return_dict=False, generator=generator)[0]

    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.cpu().permute(0, 2, 3, 1).numpy()
    image = (image * 255).round().astype("uint8")
    return [Image.fromarray(img) for img in image]

  def generateFast(self,prompt,images_per_prompt,noise=None,diffusion_steps=50,tokenizer=None,text_encoder=None):
    generater  = DiffusionPipeline.from_pretrained(
            MODEL_NAME,
            unet=self.unwrap_model(self.unet),
            variant=PRECISION,
        ).to(self.device)
    return generater(prompt,num_inference_steps=diffusion_steps,num_images_per_prompt=images_per_prompt)

  def save():
    pass
  def load():
    pass

In [12]:
configs = {
    "lr" : 1e-6 ,
    "adam_beta1" : 0.9,
    "adam_beta2" : 0.999,
    "betas":(0.9,0.999),
    "adam_weight_decay" : 1e-2,
    "adam_epsilon" : 1e-08,
    "lr_scheduler" : 'constant',
    "lr_warmup_steps": 0,
    "lr_num_cycles": 1,
    "max_train_steps":1200,
    "lr_power" : 1.0,
    "gradient_accumulation_steps": 1,
    "mixed_precision": "fp16",
    "report_to": "wandb",
    "gradient_checkpointing":True,
    "dir": "/content/dreambooth"
}




In [13]:
dreamBooth = DreamBoothPipeline(configs,False,False,accelerate=True)

*** Downloading and loading models ***


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:412: UserWarning: `log_with=wandb` was passed but no supported trackers are currently installed.
  warnings.warn(f"`log_with={log_with}` was passed but no supported trackers are currently installed.")
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
dreamBooth.train(train_dataloader)

Epoch 1/24:   0%|          | 0/51 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:1399: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with device_autocast_ctx, torch.cpu.amp.autocast(**cpu_autocast_kwargs), recompute_context:  # type: ignore[attr-defined]


Epoch 1 loss: 0.8145638423807481


Epoch 2 loss: 0.7119813608188256


Epoch 3 loss: 0.6340532285325667


Epoch 4 loss: 0.37135886123367384


Epoch 5 loss: 0.21604255207029044


Epoch 6 loss: 0.1909747758843735


Epoch 7/24:  16%|█▌        | 8/51 [00:15<01:20,  1.88s/it, loss=0.231, avg_loss=0.139]

In [19]:
images = dreamBooth.generate("A  women with blue eyes and blonde hair  wearing cap",3,tokenizer=tokenizer,text_encoder=text_encoder)

<ipython-input-15-d368098b9263>:165: FutureWarning: Accessing config attribute `in_channels` directly via 'UNet2DConditionModel' object attribute is deprecated. Please access 'in_channels' over 'UNet2DConditionModel's config object instead, e.g. 'unet.config.in_channels'.
  init_latent = torch.zeros((images_per_prompt, unet.in_channels, 768 // 8, 768 // 8), device=self.device)


tensor(981)
tensor(961)
tensor(941)
tensor(921)
tensor(901)
tensor(881)
tensor(861)
tensor(841)
tensor(821)
tensor(801)
tensor(781)
tensor(761)
tensor(741)
tensor(721)
tensor(701)
tensor(681)
tensor(661)
tensor(641)
tensor(621)
tensor(601)
tensor(581)
tensor(561)
tensor(541)
tensor(521)
tensor(501)
tensor(481)
tensor(461)
tensor(441)
tensor(421)
tensor(401)
tensor(381)
tensor(361)
tensor(341)
tensor(321)
tensor(301)
tensor(281)
tensor(261)
tensor(241)
tensor(221)
tensor(201)
tensor(181)
tensor(161)
tensor(141)
tensor(121)
tensor(101)
tensor(81)
tensor(61)
tensor(41)
tensor(21)
tensor(1)


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.69 GiB. GPU 0 has a total capacity of 14.75 GiB of which 1005.06 MiB is free. Process 307179 has 13.76 GiB memory in use. Of the allocated memory 10.39 GiB is allocated by PyTorch, and 3.24 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:

images[2]

In [18]:
generate_image = Image.fromarray((images[1] * 255).astype("uint8"))


TypeError: 'Image' object is not subscriptable

In [ ]:
generate_image

In [ ]:
from diffusers import  AutoencoderKL
from transformers import CLIPTokenizer, CLIPTextModel
import torch
from PIL import Image

def inference(model, prompt, num_steps=50, output_image_path="generated_image.png"):
        # Tokenize the input prompt
        inputs = model.tokenizer(prompt, return_tensors="pt").input_ids.to(self.device)

        # Encode the text input to get text embeddings
        text_embeddings = model.text_encoder(inputs)[0]
        # Generate latent noise for the image
        latents = torch.randn((1, self.unet.config.in_channels, 64, 64), device=self.device, dtype=self.dtype)

        # Prepare for the denoising loop
        self.scheduler.set_timesteps(num_steps)
        latents = latents * self.scheduler.init_noise_sigma

        # Denoising loop using the UNet model
        for t in self.scheduler.timesteps:
            latent_model_input = self.scheduler.scale_model_input(latents, t)
            noise_pred = self.unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
            latents = self.scheduler.step(noise_pred, t, latents).prev_sample

        # Decode the latents to pixel space using the VAE
        latents = 1 / 0.18215 * latents
        image = self.vae.decode(latents).sample

        # Convert the image to PIL format
        image = (image / 2 + 0.5).clamp(0, 1)
        image = image.cpu().permute(0, 2, 3, 1).numpy()
        image = Image.fromarray((image[0] * 255).astype("uint8"))

        # Save or display the generated image
        image.save(output_image_path)
        image.show()


In [ ]:
    sd_model.inference(prompt="A scenic landscape with mountains during sunset.")


In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "stabilityai/stable-diffusion-2"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)



In [ ]:
import os
prompts = ["A young beautiful women with medium hair with a short smile",
          "face of beatiful young indian lady with long hair dressed in saree  ",
          "A young  indian women dressed in shirt with bob hair ",
          "A beautiful women dressed formally wearing sunglases",
           "A  women with blue eyes and blonde hair dressed in jeans wearing bag",
         "A  women with brown eyes and straight hair dressed in traditional sitting in temple",

]
prompts = prompts * 10
pipe.to("cuda")
index = 0
for prompt in prompts:
  images = pipe(prompt=prompt, num_inference_steps=35, num_images_per_prompt = 8)[0]
  for i in range(len(images)):
          images[i].save(os.path.join("/content/classimgs", f"generated_{index}.png"))
          index +=1
del(pipe)


In [ ]:
  @staticmethod
  def generate_class_images(class_images_dir, n_class:int, class_prompt:str, device="cpu"):
    if not os.path.exists(class_images_dir):
      os.mkdir(class_images_dir)

    if n_class > len(os.listdir(class_images_dir)):
      print("Generating class images")

      vae = AutoencoderKL.from_pretrained(
          "madebyollin/sdxl-vae-fp16-fix",
          torch_dtype=torch.float16
      )
      pipe = DiffusionPipeline.from_pretrained(
          "stabilityai/stable-diffusion-xl-base-1.0",
          vae=vae,
          torch_dtype=torch.float16,
          variant="fp16",
          use_safetensors=True,
      )
      pipe.to("cuda");

      base_SD_pipeline = DiffusionPipeline.from_pretrained(MODEL_NAME, torch_dtype=torch.float32)

      images = base_SD_pipeline(prompt=class_prompt, num_inference_steps=25, num_images_per_prompt=n_class-len(os.listdir(class_images_dir)))
      for i in range(len(images)):
        images[i].save(os.path.join(class_images_dir, f"generated_{i}.png"))
      del(base_SD_pipeline)

    else:
      print("Class images already generated")


In [ ]:
# prompt: zip this folder /content/classimgs

!zip -r /content/classimgs.zip /content/classimgs


**ERROR** :

```
ValueError: Attempting to unscale FP16 gradients.
```

use_safetensors=True during training this causes the error

 configurations may not work well with mixed precision when using safetensors

 ```
Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward```


1) during inference of text encoder for prompt it creates a graph for backpass
2) this graph is deleted during backpass of first batch
3) so while second batch graph is not there
4) so if we inference with no grad the op of text encoder will have no graph connection so backward pas will no backprogate there

In [ ]:
# prompt: what does this do
#         latents = self.scheduler.scale_model_input(noise, t)

# It scales the input noise based on the current timestep (t) in the diffusion process.

# Let's break down what's happening:

# 1. `self.scheduler`: This refers to the scheduler object, which is responsible for managing the diffusion process. It contains methods for adding noise, removing noise, and scaling the input noise.

# 2. `self.scheduler.scale_model_input(noise, t)`: This method takes the current noise and the current timestep as input and returns a scaled version of the noise.

# The scaling is important because it helps the UNet model learn to predict the noise more effectively at different stages of the diffusion process. As the diffusion process progresses, the noise level decreases, and the model needs to be able to predict the noise more accurately at lower noise levels.

# In essence, this line of code prepares the input noise for the UNet model by scaling it based on the current timestep. This ensures that the model receives appropriate input for each stage of the denoising process.
